# Concept Resources API Test Cases

This notebook contains comprehensive test cases for the Concept Resources API endpoints.

## Prerequisites
1. Flask server running on `http://localhost:5200`
2. Database table `concept_resources` created in Supabase
3. At least one concept exists in the database

In [9]:
import requests
import json
from datetime import datetime
import pandas as pd

# Configuration
BASE_URL = "http://localhost:5200/api"

# Helper function to print formatted JSON
def print_json(data, title="Response"):
    print(f"\n{'='*60}")
    print(f"{title}")
    print('='*60)
    print(json.dumps(data, indent=2))
    print('='*60)

# Test results storage
test_results = []

def log_test(test_name, passed, details=""):
    result = {
        "test_name": test_name,
        "passed": passed,
        "details": details,
        "timestamp": datetime.now().isoformat()
    }
    test_results.append(result)
    status = "✅ PASSED" if passed else "❌ FAILED"
    print(f"\n{status}: {test_name}")
    if details:
        print(f"Details: {details}")

print("✅ Imports successful!")

✅ Imports successful!


## Setup: Get Test Data

First, we need to get existing hierarchy data (exam, subject, chapter, topic, concept) to use in our tests.

In [10]:
# Get all exams
response = requests.get(f"{BASE_URL}/hierarchy/exams")
exams = response.json()

if exams and len(exams) > 0:
    test_exam = exams[0]
    exam_id = test_exam['id']
    print(f"Using Exam: {test_exam['name']} (ID: {exam_id})")
else:
    print("⚠️ No exams found. Please create an exam first.")
    exam_id = None

Using Exam: Notebook School Exam 20251004053309 (ID: edaf8ab2-bcff-4a10-a7f2-005927724f3e)


In [11]:
# Get subjects for the exam
if exam_id:
    response = requests.get(f"{BASE_URL}/hierarchy/subjects?exam_id={exam_id}")
    subjects = response.json()
    
    if subjects and len(subjects) > 0:
        test_subject = subjects[0]
        subject_id = test_subject['id']
        print(f"Using Subject: {test_subject['name']} (ID: {subject_id})")
    else:
        print("⚠️ No subjects found. Creating a test subject...")
        response = requests.post(f"{BASE_URL}/subjects", json={
            "exam_id": exam_id,
            "name": "Test Subject for Resources",
            "description": "Auto-created for resource testing"
        })
        test_subject = response.json()[0] if isinstance(response.json(), list) else response.json()
        subject_id = test_subject['id']
        print(f"Created Subject: {test_subject['name']} (ID: {subject_id})")

⚠️ No subjects found. Creating a test subject...
Created Subject: Test Subject for Resources (ID: faf04c9d-0e58-4a45-b945-a68b799e68fe)


In [12]:
# Get chapters for the subject
if subject_id:
    response = requests.get(f"{BASE_URL}/hierarchy/chapters?subject_id={subject_id}")
    chapters = response.json()
    
    if chapters and len(chapters) > 0:
        test_chapter = chapters[0]
        chapter_id = test_chapter['id']
        print(f"Using Chapter: {test_chapter['name']} (ID: {chapter_id})")
    else:
        print("⚠️ No chapters found. Creating a test chapter...")
        response = requests.post(f"{BASE_URL}/chapters", json={
            "subject_id": subject_id,
            "name": "Test Chapter for Resources",
            "description": "Auto-created for resource testing"
        })
        test_chapter = response.json()[0] if isinstance(response.json(), list) else response.json()
        chapter_id = test_chapter['id']
        print(f"Created Chapter: {test_chapter['name']} (ID: {chapter_id})")

⚠️ No chapters found. Creating a test chapter...
Created Chapter: Test Chapter for Resources (ID: 8b208535-3682-4e4b-83be-49c644a42b09)


In [13]:
# Get topics for the chapter
if chapter_id:
    response = requests.get(f"{BASE_URL}/hierarchy/topics?chapter_id={chapter_id}")
    topics = response.json()
    
    if topics and len(topics) > 0:
        test_topic = topics[0]
        topic_id = test_topic['id']
        print(f"Using Topic: {test_topic['name']} (ID: {topic_id})")
    else:
        print("⚠️ No topics found. Creating a test topic...")
        response = requests.post(f"{BASE_URL}/topics", json={
            "chapter_id": chapter_id,
            "name": "Test Topic for Resources",
            "description": "Auto-created for resource testing"
        })
        test_topic = response.json()[0] if isinstance(response.json(), list) else response.json()
        topic_id = test_topic['id']
        print(f"Created Topic: {test_topic['name']} (ID: {topic_id})")

⚠️ No topics found. Creating a test topic...
Created Topic: Test Topic for Resources (ID: 95f24d0b-8723-49ab-80a3-60ff723cb97a)


In [14]:
# Get concepts for the topic
if topic_id:
    response = requests.get(f"{BASE_URL}/hierarchy/concepts?topic_id={topic_id}")
    concepts = response.json()
    
    if concepts and len(concepts) > 0:
        test_concept = concepts[0]
        concept_id = test_concept['id']
        print(f"Using Concept: {test_concept['name']} (ID: {concept_id})")
    else:
        print("⚠️ No concepts found. Creating a test concept...")
        response = requests.post(f"{BASE_URL}/concepts", json={
            "topic_id": topic_id,
            "name": "Test Concept for Resources",
            "description": "Auto-created for resource testing"
        })
        test_concept = response.json()[0] if isinstance(response.json(), list) else response.json()
        concept_id = test_concept['id']
        print(f"Created Concept: {test_concept['name']} (ID: {concept_id})")

print(f"\n✅ Setup Complete!")
print(f"Concept ID for testing: {concept_id}")

⚠️ No concepts found. Creating a test concept...
Created Concept: Test Concept for Resources (ID: cc479800-6ac9-4d50-b11d-57ee6ba240ed)

✅ Setup Complete!
Concept ID for testing: cc479800-6ac9-4d50-b11d-57ee6ba240ed


## Test 1: Add a Single Video Resource

In [15]:
# Test adding a video resource
video_resource = {
    "resource_type": "video",
    "title": "Introduction to Photosynthesis",
    "description": "A comprehensive video explaining the process of photosynthesis in plants",
    "url": "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
    "thumbnail_url": "https://img.youtube.com/vi/dQw4w9WgXcQ/maxresdefault.jpg",
    "duration": 600,
    "metadata": {
        "platform": "YouTube",
        "quality": "1080p",
        "language": "English",
        "subtitles": ["English", "Spanish"]
    },
    "order_index": 0
}

response = requests.post(
    f"{BASE_URL}/concepts/{concept_id}/resources",
    json=video_resource
)

if response.status_code == 201:
    video_result = response.json()
    video_resource_id = video_result['id']
    print_json(video_result, "✅ Video Resource Created")
    log_test("Add Video Resource", True, f"Resource ID: {video_resource_id}")
else:
    print(f"❌ Failed: {response.status_code}")
    print_json(response.json(), "Error Response")
    log_test("Add Video Resource", False, f"Status: {response.status_code}")
    video_resource_id = None


✅ Video Resource Created
{
  "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
  "created_at": "2025-10-04T11:19:24.820353+00:00",
  "description": "A comprehensive video explaining the process of photosynthesis in plants",
  "duration": 600,
  "file_size": null,
  "id": "72bd1481-d064-4b5b-9f8c-00acf3fa5791",
  "is_active": true,
  "metadata": {
    "language": "English",
    "platform": "YouTube",
    "quality": "1080p",
    "subtitles": [
      "English",
      "Spanish"
    ]
  },
  "order_index": 0,
  "resource_type": "video",
  "thumbnail_url": "https://img.youtube.com/vi/dQw4w9WgXcQ/maxresdefault.jpg",
  "title": "Introduction to Photosynthesis",
  "updated_at": "2025-10-04T11:19:24.820353+00:00",
  "url": "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
}

✅ PASSED: Add Video Resource
Details: Resource ID: 72bd1481-d064-4b5b-9f8c-00acf3fa5791


## Test 2: Add a 3D Model Resource

In [16]:
# Test adding a 3D model resource
model_resource = {
    "resource_type": "3d_model",
    "title": "Chloroplast 3D Model",
    "description": "Interactive 3D model of a chloroplast showing internal structures",
    "url": "https://example.com/models/chloroplast.gltf",
    "thumbnail_url": "https://example.com/thumbnails/chloroplast.png",
    "file_size": 5242880,
    "metadata": {
        "format": "GLTF",
        "polygons": 50000,
        "animated": True,
        "textures": True
    },
    "order_index": 1
}

response = requests.post(
    f"{BASE_URL}/concepts/{concept_id}/resources",
    json=model_resource
)

if response.status_code == 201:
    model_result = response.json()
    model_resource_id = model_result['id']
    print_json(model_result, "✅ 3D Model Resource Created")
    log_test("Add 3D Model Resource", True, f"Resource ID: {model_resource_id}")
else:
    print(f"❌ Failed: {response.status_code}")
    print_json(response.json(), "Error Response")
    log_test("Add 3D Model Resource", False, f"Status: {response.status_code}")
    model_resource_id = None


✅ 3D Model Resource Created
{
  "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
  "created_at": "2025-10-04T11:19:34.925612+00:00",
  "description": "Interactive 3D model of a chloroplast showing internal structures",
  "duration": null,
  "file_size": 5242880,
  "id": "981272ed-db48-4b5d-ad59-6f57621653a9",
  "is_active": true,
  "metadata": {
    "animated": true,
    "format": "GLTF",
    "polygons": 50000,
    "textures": true
  },
  "order_index": 1,
  "resource_type": "3d_model",
  "thumbnail_url": "https://example.com/thumbnails/chloroplast.png",
  "title": "Chloroplast 3D Model",
  "updated_at": "2025-10-04T11:19:34.925612+00:00",
  "url": "https://example.com/models/chloroplast.gltf"
}

✅ PASSED: Add 3D Model Resource
Details: Resource ID: 981272ed-db48-4b5d-ad59-6f57621653a9


## Test 3: Add a Virtual Lab Resource

In [17]:
# Test adding a virtual lab resource
lab_resource = {
    "resource_type": "virtual_lab",
    "title": "Photosynthesis Lab Simulator",
    "description": "Interactive virtual lab to experiment with photosynthesis variables",
    "url": "https://phet.colorado.edu/en/simulation/photosynthesis",
    "metadata": {
        "platform": "PhET",
        "interactive": True,
        "experiments": ["light_intensity", "co2_levels", "temperature"],
        "difficulty": "intermediate",
        "supported_browsers": ["Chrome", "Firefox", "Safari"]
    },
    "order_index": 2
}

response = requests.post(
    f"{BASE_URL}/concepts/{concept_id}/resources",
    json=lab_resource
)

if response.status_code == 201:
    lab_result = response.json()
    lab_resource_id = lab_result['id']
    print_json(lab_result, "✅ Virtual Lab Resource Created")
    log_test("Add Virtual Lab Resource", True, f"Resource ID: {lab_resource_id}")
else:
    print(f"❌ Failed: {response.status_code}")
    print_json(response.json(), "Error Response")
    log_test("Add Virtual Lab Resource", False, f"Status: {response.status_code}")
    lab_resource_id = None


✅ Virtual Lab Resource Created
{
  "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
  "created_at": "2025-10-04T11:19:41.247888+00:00",
  "description": "Interactive virtual lab to experiment with photosynthesis variables",
  "duration": null,
  "file_size": null,
  "id": "2a85c382-4037-4d7e-add6-581acbade1f7",
  "is_active": true,
  "metadata": {
    "difficulty": "intermediate",
    "experiments": [
      "light_intensity",
      "co2_levels",
      "temperature"
    ],
    "interactive": true,
    "platform": "PhET",
    "supported_browsers": [
      "Chrome",
      "Firefox",
      "Safari"
    ]
  },
  "order_index": 2,
  "resource_type": "virtual_lab",
  "thumbnail_url": null,
  "title": "Photosynthesis Lab Simulator",
  "updated_at": "2025-10-04T11:19:41.247888+00:00",
  "url": "https://phet.colorado.edu/en/simulation/photosynthesis"
}

✅ PASSED: Add Virtual Lab Resource
Details: Resource ID: 2a85c382-4037-4d7e-add6-581acbade1f7


## Test 4: Add Image and Animation Resources

In [18]:
# Test adding an image resource
image_resource = {
    "resource_type": "image",
    "title": "Photosynthesis Diagram",
    "description": "Detailed diagram showing the light and dark reactions",
    "url": "https://example.com/images/photosynthesis-diagram.png",
    "metadata": {
        "resolution": "1920x1080",
        "format": "PNG",
        "license": "CC-BY-4.0"
    },
    "order_index": 3
}

response = requests.post(
    f"{BASE_URL}/concepts/{concept_id}/resources",
    json=image_resource
)

image_resource_id = None
if response.status_code == 201:
    image_result = response.json()
    image_resource_id = image_result['id']
    print_json(image_result, "✅ Image Resource Created")
    log_test("Add Image Resource", True, f"Resource ID: {image_resource_id}")
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Add Image Resource", False, f"Status: {response.status_code}")

# Test adding an animation resource
animation_resource = {
    "resource_type": "animation",
    "title": "Photosynthesis Process Animation",
    "description": "Animated visualization of electron transport chain",
    "url": "https://example.com/animations/photosynthesis.mp4",
    "duration": 120,
    "metadata": {
        "format": "MP4",
        "fps": 60,
        "loopable": True
    },
    "order_index": 4
}

response = requests.post(
    f"{BASE_URL}/concepts/{concept_id}/resources",
    json=animation_resource
)

animation_resource_id = None
if response.status_code == 201:
    animation_result = response.json()
    animation_resource_id = animation_result['id']
    print_json(animation_result, "✅ Animation Resource Created")
    log_test("Add Animation Resource", True, f"Resource ID: {animation_resource_id}")
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Add Animation Resource", False, f"Status: {response.status_code}")


✅ Image Resource Created
{
  "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
  "created_at": "2025-10-04T11:19:46.402462+00:00",
  "description": "Detailed diagram showing the light and dark reactions",
  "duration": null,
  "file_size": null,
  "id": "0da182a7-549a-4b41-81eb-29d48b76e8c0",
  "is_active": true,
  "metadata": {
    "format": "PNG",
    "license": "CC-BY-4.0",
    "resolution": "1920x1080"
  },
  "order_index": 3,
  "resource_type": "image",
  "thumbnail_url": null,
  "title": "Photosynthesis Diagram",
  "updated_at": "2025-10-04T11:19:46.402462+00:00",
  "url": "https://example.com/images/photosynthesis-diagram.png"
}

✅ PASSED: Add Image Resource
Details: Resource ID: 0da182a7-549a-4b41-81eb-29d48b76e8c0

✅ Animation Resource Created
{
  "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
  "created_at": "2025-10-04T11:19:46.725469+00:00",
  "description": "Animated visualization of electron transport chain",
  "duration": 120,
  "file_size": null,
  "id": "84

## Test 5: Bulk Add Resources

In [19]:
# Test bulk adding resources
bulk_resources = {
    "resources": [
        {
            "resource_type": "pdf",
            "title": "Photosynthesis Study Guide",
            "description": "Comprehensive PDF guide covering all aspects",
            "url": "https://example.com/docs/photosynthesis-guide.pdf",
            "file_size": 2048000,
            "metadata": {
                "pages": 25,
                "language": "English"
            }
        },
        {
            "resource_type": "interactive",
            "title": "Interactive Quiz",
            "description": "Test your knowledge with this interactive quiz",
            "url": "https://example.com/quiz/photosynthesis",
            "metadata": {
                "questions": 20,
                "time_limit": 30,
                "difficulty": "medium"
            }
        },
        {
            "resource_type": "video",
            "title": "Advanced Photosynthesis Concepts",
            "url": "https://youtube.com/advanced-photo",
            "duration": 900
        }
    ]
}

response = requests.post(
    f"{BASE_URL}/concepts/{concept_id}/resources/bulk",
    json=bulk_resources
)

if response.status_code == 201:
    bulk_result = response.json()
    print_json(bulk_result, "✅ Bulk Resources Created")
    log_test("Bulk Add Resources", True, f"Created {bulk_result['created_count']} resources")
else:
    print(f"❌ Failed: {response.status_code}")
    print_json(response.json(), "Error Response")
    log_test("Bulk Add Resources", False, f"Status: {response.status_code}")


✅ Bulk Resources Created
{
  "created_count": 3,
  "resources": [
    {
      "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
      "created_at": "2025-10-04T11:19:51.069892+00:00",
      "description": "Comprehensive PDF guide covering all aspects",
      "duration": null,
      "file_size": 2048000,
      "id": "c5944472-b723-4a84-8ccb-3ea1417e58cf",
      "is_active": true,
      "metadata": {
        "language": "English",
        "pages": 25
      },
      "order_index": 0,
      "resource_type": "pdf",
      "thumbnail_url": null,
      "title": "Photosynthesis Study Guide",
      "updated_at": "2025-10-04T11:19:51.069892+00:00",
      "url": "https://example.com/docs/photosynthesis-guide.pdf"
    },
    {
      "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
      "created_at": "2025-10-04T11:19:51.069892+00:00",
      "description": "Test your knowledge with this interactive quiz",
      "duration": null,
      "file_size": null,
      "id": "84192f07-5d9b-446d-886

## Test 6: Get All Resources for a Concept

In [20]:
# Get all resources for the concept
response = requests.get(f"{BASE_URL}/concepts/{concept_id}/resources")

if response.status_code == 200:
    resources_result = response.json()
    print_json(resources_result, "✅ All Resources for Concept")
    log_test("Get All Concept Resources", True, f"Found {resources_result['count']} resources")
    
    # Display as DataFrame
    if resources_result['resources']:
        df = pd.DataFrame(resources_result['resources'])
        print("\n📊 Resources Summary:")
        print(df[['resource_type', 'title', 'order_index', 'is_active']].to_string())
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Get All Concept Resources", False, f"Status: {response.status_code}")


✅ All Resources for Concept
{
  "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
  "concept_name": "Test Concept for Resources",
  "count": 8,
  "resources": [
    {
      "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
      "created_at": "2025-10-04T11:19:51.069892+00:00",
      "description": "Comprehensive PDF guide covering all aspects",
      "duration": null,
      "file_size": 2048000,
      "id": "c5944472-b723-4a84-8ccb-3ea1417e58cf",
      "is_active": true,
      "metadata": {
        "language": "English",
        "pages": 25
      },
      "order_index": 0,
      "resource_type": "pdf",
      "thumbnail_url": null,
      "title": "Photosynthesis Study Guide",
      "updated_at": "2025-10-04T11:19:51.069892+00:00",
      "url": "https://example.com/docs/photosynthesis-guide.pdf"
    },
    {
      "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
      "created_at": "2025-10-04T11:19:24.820353+00:00",
      "description": "A comprehensive video explaining t

## Test 7: Filter Resources by Type

In [21]:
# Get only video resources
response = requests.get(f"{BASE_URL}/concepts/{concept_id}/resources?resource_type=video")

if response.status_code == 200:
    videos = response.json()
    print_json(videos, "✅ Video Resources Only")
    log_test("Filter by Video Type", True, f"Found {videos['count']} videos")
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Filter by Video Type", False, f"Status: {response.status_code}")

# Get only 3D model resources
response = requests.get(f"{BASE_URL}/concepts/{concept_id}/resources?resource_type=3d_model")

if response.status_code == 200:
    models = response.json()
    print_json(models, "✅ 3D Model Resources Only")
    log_test("Filter by 3D Model Type", True, f"Found {models['count']} models")
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Filter by 3D Model Type", False, f"Status: {response.status_code}")


✅ Video Resources Only
{
  "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
  "concept_name": "Test Concept for Resources",
  "count": 2,
  "resources": [
    {
      "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
      "created_at": "2025-10-04T11:19:24.820353+00:00",
      "description": "A comprehensive video explaining the process of photosynthesis in plants",
      "duration": 600,
      "file_size": null,
      "id": "72bd1481-d064-4b5b-9f8c-00acf3fa5791",
      "is_active": true,
      "metadata": {
        "language": "English",
        "platform": "YouTube",
        "quality": "1080p",
        "subtitles": [
          "English",
          "Spanish"
        ]
      },
      "order_index": 0,
      "resource_type": "video",
      "thumbnail_url": "https://img.youtube.com/vi/dQw4w9WgXcQ/maxresdefault.jpg",
      "title": "Introduction to Photosynthesis",
      "updated_at": "2025-10-04T11:19:24.820353+00:00",
      "url": "https://www.youtube.com/watch?v=dQw4w9WgXcQ"

## Test 8: Get a Specific Resource by ID

In [22]:
# Get specific resource by ID
if video_resource_id:
    response = requests.get(f"{BASE_URL}/resources/{video_resource_id}")
    
    if response.status_code == 200:
        resource_detail = response.json()
        print_json(resource_detail, "✅ Specific Resource Retrieved")
        log_test("Get Resource by ID", True, f"Resource: {resource_detail['title']}")
    else:
        print(f"❌ Failed: {response.status_code}")
        log_test("Get Resource by ID", False, f"Status: {response.status_code}")
else:
    print("⚠️ Skipping: No video resource ID available")
    log_test("Get Resource by ID", False, "No resource ID")


✅ Specific Resource Retrieved
{
  "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
  "created_at": "2025-10-04T11:19:24.820353+00:00",
  "description": "A comprehensive video explaining the process of photosynthesis in plants",
  "duration": 600,
  "file_size": null,
  "id": "72bd1481-d064-4b5b-9f8c-00acf3fa5791",
  "is_active": true,
  "metadata": {
    "language": "English",
    "platform": "YouTube",
    "quality": "1080p",
    "subtitles": [
      "English",
      "Spanish"
    ]
  },
  "order_index": 0,
  "resource_type": "video",
  "thumbnail_url": "https://img.youtube.com/vi/dQw4w9WgXcQ/maxresdefault.jpg",
  "title": "Introduction to Photosynthesis",
  "updated_at": "2025-10-04T11:19:24.820353+00:00",
  "url": "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
}

✅ PASSED: Get Resource by ID
Details: Resource: Introduction to Photosynthesis


## Test 9: Update a Resource

In [23]:
# Update resource
if video_resource_id:
    update_data = {
        "title": "Introduction to Photosynthesis - UPDATED",
        "description": "Updated description with more details",
        "duration": 720,
        "metadata": {
            "platform": "YouTube",
            "quality": "4K",
            "language": "English",
            "subtitles": ["English", "Spanish", "French"],
            "updated": True
        }
    }
    
    response = requests.put(
        f"{BASE_URL}/resources/{video_resource_id}",
        json=update_data
    )
    
    if response.status_code == 200:
        updated_resource = response.json()
        print_json(updated_resource, "✅ Resource Updated")
        log_test("Update Resource", True, f"Updated: {updated_resource['title']}")
    else:
        print(f"❌ Failed: {response.status_code}")
        print_json(response.json(), "Error Response")
        log_test("Update Resource", False, f"Status: {response.status_code}")
else:
    print("⚠️ Skipping: No video resource ID available")
    log_test("Update Resource", False, "No resource ID")


✅ Resource Updated
{
  "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
  "created_at": "2025-10-04T11:19:24.820353+00:00",
  "description": "Updated description with more details",
  "duration": 720,
  "file_size": null,
  "id": "72bd1481-d064-4b5b-9f8c-00acf3fa5791",
  "is_active": true,
  "metadata": {
    "language": "English",
    "platform": "YouTube",
    "quality": "4K",
    "subtitles": [
      "English",
      "Spanish",
      "French"
    ],
    "updated": true
  },
  "order_index": 0,
  "resource_type": "video",
  "thumbnail_url": "https://img.youtube.com/vi/dQw4w9WgXcQ/maxresdefault.jpg",
  "title": "Introduction to Photosynthesis - UPDATED",
  "updated_at": "2025-10-04T11:19:24.820353+00:00",
  "url": "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
}

✅ PASSED: Update Resource
Details: Updated: Introduction to Photosynthesis - UPDATED


## Test 10: Get Topic Resources (All Concepts)

In [24]:
# Get all resources for the topic (grouped by concept)
response = requests.get(f"{BASE_URL}/topics/{topic_id}/resources")

if response.status_code == 200:
    topic_resources = response.json()
    print_json(topic_resources, "✅ Topic Resources (Grouped by Concept)")
    log_test("Get Topic Resources", True, f"Total: {topic_resources['total_resources']} resources")
    
    # Display summary
    print("\n📊 Topic Resources Summary:")
    for concept_data in topic_resources['concepts']:
        print(f"\nConcept: {concept_data['concept_name']}")
        print(f"Resource Count: {concept_data['resource_count']}")
        if concept_data['resources']:
            for res in concept_data['resources']:
                print(f"  - [{res['resource_type']}] {res['title']}")
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Get Topic Resources", False, f"Status: {response.status_code}")


✅ Topic Resources (Grouped by Concept)
{
  "concepts": [
    {
      "concept_description": "Auto-created for resource testing",
      "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
      "concept_name": "Test Concept for Resources",
      "resource_count": 8,
      "resources": [
        {
          "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
          "created_at": "2025-10-04T11:19:51.069892+00:00",
          "description": "Comprehensive PDF guide covering all aspects",
          "duration": null,
          "file_size": 2048000,
          "id": "c5944472-b723-4a84-8ccb-3ea1417e58cf",
          "is_active": true,
          "metadata": {
            "language": "English",
            "pages": 25
          },
          "order_index": 0,
          "resource_type": "pdf",
          "thumbnail_url": null,
          "title": "Photosynthesis Study Guide",
          "updated_at": "2025-10-04T11:19:51.069892+00:00",
          "url": "https://example.com/docs/photosynthesis-

## Test 11: Soft Delete a Resource

In [25]:
# Soft delete (set is_active = false)
if image_resource_id:
    response = requests.delete(f"{BASE_URL}/resources/{image_resource_id}")
    
    if response.status_code == 200:
        delete_result = response.json()
        print_json(delete_result, "✅ Resource Soft Deleted")
        log_test("Soft Delete Resource", True, "Resource marked as inactive")
        
        # Verify it's not returned in active resources
        verify_response = requests.get(f"{BASE_URL}/concepts/{concept_id}/resources?is_active=true")
        active_resources = verify_response.json()
        
        is_hidden = all(r['id'] != image_resource_id for r in active_resources['resources'])
        if is_hidden:
            print("✅ Verified: Soft-deleted resource is hidden from active list")
        else:
            print("⚠️ Warning: Soft-deleted resource still appears in active list")
    else:
        print(f"❌ Failed: {response.status_code}")
        log_test("Soft Delete Resource", False, f"Status: {response.status_code}")
else:
    print("⚠️ Skipping: No image resource ID available")
    log_test("Soft Delete Resource", False, "No resource ID")


✅ Resource Soft Deleted
{
  "message": "Resource deleted successfully",
  "success": true
}

✅ PASSED: Soft Delete Resource
Details: Resource marked as inactive
✅ Verified: Soft-deleted resource is hidden from active list


## Test 12: Get Inactive Resources

In [26]:
# Get inactive resources
response = requests.get(f"{BASE_URL}/concepts/{concept_id}/resources?is_active=false")

if response.status_code == 200:
    inactive_resources = response.json()
    print_json(inactive_resources, "✅ Inactive Resources")
    log_test("Get Inactive Resources", True, f"Found {inactive_resources['count']} inactive")
else:
    print(f"❌ Failed: {response.status_code}")
    log_test("Get Inactive Resources", False, f"Status: {response.status_code}")


✅ Inactive Resources
{
  "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
  "concept_name": "Test Concept for Resources",
  "count": 1,
  "resources": [
    {
      "concept_id": "cc479800-6ac9-4d50-b11d-57ee6ba240ed",
      "created_at": "2025-10-04T11:19:46.402462+00:00",
      "description": "Detailed diagram showing the light and dark reactions",
      "duration": null,
      "file_size": null,
      "id": "0da182a7-549a-4b41-81eb-29d48b76e8c0",
      "is_active": false,
      "metadata": {
        "format": "PNG",
        "license": "CC-BY-4.0",
        "resolution": "1920x1080"
      },
      "order_index": 3,
      "resource_type": "image",
      "thumbnail_url": null,
      "title": "Photosynthesis Diagram",
      "updated_at": "2025-10-04T11:19:46.402462+00:00",
      "url": "https://example.com/images/photosynthesis-diagram.png"
    }
  ]
}

✅ PASSED: Get Inactive Resources
Details: Found 1 inactive


## Test 13: Error Handling - Invalid Resource Type

In [27]:
# Test with invalid resource type
invalid_resource = {
    "resource_type": "invalid_type",
    "title": "Test Invalid Type",
    "url": "https://example.com"
}

response = requests.post(
    f"{BASE_URL}/concepts/{concept_id}/resources",
    json=invalid_resource
)

if response.status_code == 400:
    error_result = response.json()
    print_json(error_result, "✅ Correct Error Response for Invalid Type")
    log_test("Invalid Resource Type Error", True, "Properly rejected invalid type")
else:
    print(f"⚠️ Unexpected status: {response.status_code}")
    log_test("Invalid Resource Type Error", False, f"Expected 400, got {response.status_code}")


✅ Correct Error Response for Invalid Type
{
  "error": "Invalid resource_type. Must be one of: ['video', 'image', '3d_model', 'animation', 'virtual_lab', 'pdf', 'interactive']"
}

✅ PASSED: Invalid Resource Type Error
Details: Properly rejected invalid type


## Test 14: Error Handling - Missing Required Fields

In [28]:
# Test with missing required fields
incomplete_resource = {
    "resource_type": "video"
    # Missing title and url
}

response = requests.post(
    f"{BASE_URL}/concepts/{concept_id}/resources",
    json=incomplete_resource
)

if response.status_code == 400:
    error_result = response.json()
    print_json(error_result, "✅ Correct Error Response for Missing Fields")
    log_test("Missing Fields Error", True, "Properly rejected incomplete data")
else:
    print(f"⚠️ Unexpected status: {response.status_code}")
    log_test("Missing Fields Error", False, f"Expected 400, got {response.status_code}")


✅ Correct Error Response for Missing Fields
{
  "error": "resource_type, title, and url are required"
}

✅ PASSED: Missing Fields Error
Details: Properly rejected incomplete data


## Test 15: Error Handling - Invalid Concept ID

In [29]:
# Test with non-existent concept ID
fake_concept_id = "00000000-0000-0000-0000-000000000000"

valid_resource = {
    "resource_type": "video",
    "title": "Test Video",
    "url": "https://example.com"
}

response = requests.post(
    f"{BASE_URL}/concepts/{fake_concept_id}/resources",
    json=valid_resource
)

if response.status_code == 404:
    error_result = response.json()
    print_json(error_result, "✅ Correct Error Response for Invalid Concept")
    log_test("Invalid Concept ID Error", True, "Properly rejected invalid concept")
else:
    print(f"⚠️ Unexpected status: {response.status_code}")
    log_test("Invalid Concept ID Error", False, f"Expected 404, got {response.status_code}")


✅ Correct Error Response for Invalid Concept
{
  "error": "Concept not found"
}

✅ PASSED: Invalid Concept ID Error
Details: Properly rejected invalid concept


## Test 16: Permanent Delete

In [30]:
# Permanent delete
if animation_resource_id:
    response = requests.delete(f"{BASE_URL}/resources/{animation_resource_id}/permanent")
    
    if response.status_code == 200:
        delete_result = response.json()
        print_json(delete_result, "✅ Resource Permanently Deleted")
        log_test("Permanent Delete Resource", True, "Resource removed from database")
        
        # Verify it's gone
        verify_response = requests.get(f"{BASE_URL}/resources/{animation_resource_id}")
        if verify_response.status_code == 404:
            print("✅ Verified: Resource no longer exists")
        else:
            print("⚠️ Warning: Resource still accessible after deletion")
    else:
        print(f"❌ Failed: {response.status_code}")
        log_test("Permanent Delete Resource", False, f"Status: {response.status_code}")
else:
    print("⚠️ Skipping: No animation resource ID available")
    log_test("Permanent Delete Resource", False, "No resource ID")


✅ Resource Permanently Deleted
{
  "message": "Resource permanently deleted",
  "success": true
}

✅ PASSED: Permanent Delete Resource
Details: Resource removed from database
✅ Verified: Resource no longer exists


## Test Summary

In [31]:
# Display test results summary
print("\n" + "="*60)
print("TEST SUMMARY")
print("="*60)

df_results = pd.DataFrame(test_results)
total_tests = len(df_results)
passed_tests = len(df_results[df_results['passed'] == True])
failed_tests = total_tests - passed_tests

print(f"\nTotal Tests: {total_tests}")
print(f"✅ Passed: {passed_tests}")
print(f"❌ Failed: {failed_tests}")
print(f"Success Rate: {(passed_tests/total_tests*100):.1f}%\n")

print("\nDetailed Results:")
print(df_results[['test_name', 'passed', 'details']].to_string(index=False))

# Show failed tests
failed_df = df_results[df_results['passed'] == False]
if len(failed_df) > 0:
    print("\n⚠️ Failed Tests:")
    for idx, row in failed_df.iterrows():
        print(f"  - {row['test_name']}: {row['details']}")
else:
    print("\n🎉 All tests passed!")


TEST SUMMARY

Total Tests: 18
✅ Passed: 18
❌ Failed: 0
Success Rate: 100.0%


Detailed Results:
                  test_name  passed                                           details
         Add Video Resource    True Resource ID: 72bd1481-d064-4b5b-9f8c-00acf3fa5791
      Add 3D Model Resource    True Resource ID: 981272ed-db48-4b5d-ad59-6f57621653a9
   Add Virtual Lab Resource    True Resource ID: 2a85c382-4037-4d7e-add6-581acbade1f7
         Add Image Resource    True Resource ID: 0da182a7-549a-4b41-81eb-29d48b76e8c0
     Add Animation Resource    True Resource ID: 84a65fa2-675e-4973-b888-2e6b0f00eb22
         Bulk Add Resources    True                               Created 3 resources
  Get All Concept Resources    True                                 Found 8 resources
       Filter by Video Type    True                                    Found 2 videos
    Filter by 3D Model Type    True                                    Found 1 models
         Get Resource by ID    True        

## Resource Statistics

In [32]:
# Get final resource count and statistics
response = requests.get(f"{BASE_URL}/concepts/{concept_id}/resources")

if response.status_code == 200:
    final_resources = response.json()
    
    print("\n" + "="*60)
    print("RESOURCE STATISTICS")
    print("="*60)
    print(f"\nConcept: {final_resources['concept_name']}")
    print(f"Total Resources: {final_resources['count']}")
    
    if final_resources['resources']:
        df = pd.DataFrame(final_resources['resources'])
        
        # Count by type
        print("\nResources by Type:")
        type_counts = df['resource_type'].value_counts()
        for rtype, count in type_counts.items():
            print(f"  {rtype}: {count}")
        
        # Show resource list
        print("\n📋 All Resources:")
        print(df[['resource_type', 'title', 'is_active', 'order_index']].to_string(index=False))
else:
    print(f"❌ Could not fetch statistics: {response.status_code}")


RESOURCE STATISTICS

Concept: Test Concept for Resources
Total Resources: 6

Resources by Type:
  video: 2
  pdf: 1
  3d_model: 1
  interactive: 1
  virtual_lab: 1

📋 All Resources:
resource_type                                    title  is_active  order_index
        video Introduction to Photosynthesis - UPDATED       True            0
          pdf               Photosynthesis Study Guide       True            0
     3d_model                     Chloroplast 3D Model       True            1
  interactive                         Interactive Quiz       True            1
  virtual_lab             Photosynthesis Lab Simulator       True            2
        video         Advanced Photosynthesis Concepts       True            2


## Cleanup (Optional)

Uncomment and run this cell to clean up all test resources

In [ ]:
# # CLEANUP - Uncomment to delete all test resources
# response = requests.get(f"{BASE_URL}/concepts/{concept_id}/resources?is_active=true")
# if response.status_code == 200:
#     resources = response.json()['resources']
#     print(f"Deleting {len(resources)} resources...")
    
#     for resource in resources:
#         del_response = requests.delete(f"{BASE_URL}/resources/{resource['id']}/permanent")
#         if del_response.status_code == 200:
#             print(f"✅ Deleted: {resource['title']}")
#         else:
#             print(f"❌ Failed to delete: {resource['title']}")
    
#     print("\n✅ Cleanup complete!")
# else:
#     print("❌ Could not fetch resources for cleanup")